<a href="https://colab.research.google.com/github/jskaza/nfl-big-data-bowl-2023/blob/master/paper_outline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Re**current **P**ressure **P**robabilities (RePP) to Predict Pass Rusher Impact as Plays Develop

**Jon Skaza & Matt Guthrie**



> A quarterback has never completed a pass when he was flat on his back…Great pass coverage is a direct result of a great pass rush, and a great pass rush is simply a relentless desire to get to the QB        

\- Buddy Ryan<sup>1</sup>

## Introduction

Aside from Quarterbacks (QB), elite pass rushers–Defensive Ends (DE), Outside Linebackers (OLB), or, simply, Edge Rushers (EDGE)--are arguably the most coveted talents in the NFL. In fact, in each of the last 9 NFL Drafts, the #1 overall pick has either been a QB (6) or a DE (3). If we instead consider the top 2 overall picks during the same timespan, the breakdown is QB (10) DE (6), LT (1), and RB (1)$.^2$ Simply put, NFL teams place a large amount of stock in finding and evaluating pass rush talent.

In this project, we introduce an analytical framework, which we coin *Recurrent Pressure Probabilities (RePP)* that we hope can provide more depth to the often-cited count statistics *Sacks (SK)*, *QB Knockdowns (QBKD)*, *QB Hurries (HRRY)*, or when considered jointly, Pressures (PRSS). Specifically, we aim to provide two novel contributions through our approach:

1. Near “real-time” probabilities of getting a *PRSS* for each rusher throughout the course of a play

2. Identifying large “probability accelerating moments” (PAM)--moments during a player’s pass rush that led to a significant increase in the probability of obtaining a PRSS

## Methods

Our motivating factor was to leverage the inherently sequential nature of the Next Gen Stats player tracking data to learn behaviors with predictive power with respect to PRSS$.^3$ We present a long short-term memory (LSTM) artificial neural network (ANN) designed to process and learn from sequences of data via feedback connections. To illustrate the power of leveraging the sequences, we compare the LSTM approach to a “naive” logistic regression model that did not account for temporal relations in the data.

### Data

Using the player tracking data, Pro Football Focus scouting data, and general NFL game information, we create a feature matrix, $X$, with dimensionality $R \times T \times P$, where $R$ represents the number of pass rushes in the dataset at the *player-level*, $T$ represents the number of frames observed in the longest play in the data, and $P$ corresponds to the number of predictors$.^4$ Some predictors vary frame-to-frame (e.g., the pass rusher's $x$ and $y$), while others remain constant throughout the course of the play (e.g., $Quarter$). The outcome, $y$, is a boolean vector indicating a *PRSS* on the pass rush for player $i$ on passing play $s$ in game $g$.

$$
y = 
\begin{bmatrix}
y_{1,1,1} \\
y_{1,1,2} \\
y_{g,s,i} \\
\vdots
\end{bmatrix}
\quad
X = 
\begin{bmatrix}
\begin{bmatrix}
x_{1,1,1} & ... & x_{1,1,P} \\
& \vdots & \\
x_{1,T,1} & ... & x_{1,T,P} \\
\end{bmatrix} \\
\vdots\\
\begin{bmatrix}
x_{R,1,1} & ... & x_{R,1,P} \\
& \vdots & \\
x_{R,T,1} & ... & x_{R,T,P} \\
\end{bmatrix} \\
\end{bmatrix}
$$

Further, we implement two sampling methods on $X$ and $y$. First, we oversample cases of $PRSS = 1$ to create a balanced dataset. Second, we augment the dataset with random sub-sequences from each play. This reflects both objectives described in the introduction; to provide probabilities of a play's outcome at different stages of a play, we want the model to observe partial plays during training.

The table below summarizes the *frame-level* features included for each unique individual pass rush.

|Feature Category|Features|
--------|-----------|
| Coordinates | Pass Rusher, LT, LG, C, RG, RT, QB, Ball |
| Situational | Quarter, Down, Yards to Go, Absolute Yard Line, Score Difference |
| Speed | Pass Rusher, QB |
| Derived | Pass Rusher Dist from QB, QB in Tackle Box, # Blockers 


### "Naive" Logistic Regression

As mentioned above, we train a logistic regression as a baseline comparison model that does not accunt for temporality. We thus remove a dimension from $X$. $X_{logistic}$ becomes $R \times (T*P)$.

### LSTM

The ANN consists of three layers: a masking layer, a LSTM layer, and a dense, fully-connected layer.

The masking layer is used to mask values from time steps in plays with a frame count $< T$.

As mentioned. the LSTM layer is useful for processing sequential data, as it is able to retain information about past events and use it to inform processing of subsequent events. The LSTM layer in our model has 64 units and an input shape of $T \times P$.

The final layer is a dense, fully-connected layer with 1 unit and a sigmoid activation function. The sigmoid activation function maps the output to a value between 0 and 1, which can be interpreted as a probability.

The model was trained on 80% of the dataset using the Adam optimization algorithm. The remaing 20% was used as a holdout set.


## Results

As expected, the LSTM, which was able to learn from sequences of player movement, outperformed the "naive logistic regression model.



## Example Usage

We can use the trained LSTM network to output Recurrent Pressure Probabilities (RePP) on select plays.


Additionally, we can view a specific play in which the probability of a sack jumped almost instaneously.

## Appendix

[1] https://profootballtalk.nbcsports.com/2016/06/29/buddy-ryans-philosophy-quarterbacks-must-be-punished/

[2] https://www.pro-football-reference.com/draft/

[3] https://nextgenstats.nfl.com/

[4] https://www.pff.com/


*Code is available on [GitHub](https://github.com/jskaza/nfl-big-data-bowl-2023)*

 
https://openaccess.thecvf.com/content/WACV2021/papers/Hauri_Multi-Modal_Trajectory_Prediction_of_NBA_Players_WACV_2021_paper.pdfOur work methods are loosely based 
Baseline model
Logistic w/ first frame, game state
Insights from Tracking data
LSTM
Transformer
- Example Use Cases
 - Simple examples of progressing through a play (the heating up)
 - Finding "inflection points" - moment when probability jumped, skill expression
 - Animations and/or annotated videos
- Discussion
